## Geopandas

In this section, we will cover the basics of [Geopandas](https://geopandas.org/), a Python library to interact with geospatial vector data.

Geopandas provides an easy-to-use interface to vector data sets. It combines the capabilities of pandas, the data analysis package we got to know in the Geo-Python course, with the geometry handling functionality of [shapely](https://shapely.readthedocs.io/en/stable/manual.html), the geo-spatial file format support of [fiona](https://fiona.readthedocs.io/en/latest/manual.html) and the [map projection libraries of pyproj](https://pyproj4.github.io/pyproj/stable/).

The main data structures in geopandas are <span style="color: red;">GeoDataFrames</span> and <span style="color: red;">GeoSeries</span>. They extend the functionality of <span style="color: red;">pandas.DataFrames</span> and <span style="color: red;">pandas.Series</span>. This means that we can use all our pandas skills also when we work with geopandas!

Let's start with a land use polygon of Värmland!

In [ ]:
 #location (directory) of the notebook
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

In [ ]:
import geopandas
data_set = geopandas.read_file(DATA_DIRECTORY / "Landusepolygon.shp")
data_set.plot()

In [ ]:
data_set.head(5)

There is one key difference between pandas’s data frames and geopandas’ <span style="color: red;">GeoDataFrames</span>: a GeoDataFrame contains an additional column for geometries. By default, the name of this column is geometry, and it is a <span style="color: red;">GeoSeries</span> that contains the geometries (points, lines, polygons, …) as shapely.geometry objects.

First, check the data type of the read data set:

In [ ]:
type(data_set)

In [ ]:
data_set.columns

Let's keep only some of these columns 

In [ ]:
data_set = data_set[['KKOD', 'KATEGORI', "geometry"]]

In [ ]:
data_set

Let's rename the columns to make them more readable with  <span style="color: red;">dataframe.rename</span>

In [ ]:
data_set = data_set.rename(
    columns={
        "KKOD": "ID_CLASS",
        "KATEGORI": "CLASS_NAME"
    }
)

In [ ]:
data_set.head()

### Explore the data set in a map:


In [ ]:
data_set.plot()

Plotting a quick is easier than it seems! The default option of geopandas covers the whole extend of the spatial data file.

Let's investigate some more of the data in this geodata frame. We can select the first 5 geometries of this column.

In [ ]:
data_set.geometry.head()

We can see that they are polygons described with Well-Known Text (WKT) strings, in fact, <span style="color: red;">shapely.geometry</span> objects! Let's try to plot the first polygon. To do so, the shapely library is used, as you are familiar from the first lecture.

In [ ]:
# The value of the column `geometry` in row 4:
data_set.at[4, "geometry"]

Let's print the area of this polygon.

In [ ]:
# Print information about the area 
print(f"Area: {round(data_set.at[4, 'geometry'].area)} m².")

Here, we know the coordinate reference system (CRS) of the input data set. The CRS also defines the unit of measurement (in our case, metres). That’s why we can print the computed area including an area measurement unit (square metres). Let’s do the same for multiple rows, and explore different options of how to. First, use the reliable and tried iterrows() pattern:

In [ ]:
# Iterate over the first 5 rows of the data set
for index, row in data_set[:5].iterrows():
    polygon_area = row["geometry"].area
    print(f"The polygon in row {index} has a surface area of {polygon_area:0.1f} m².")

As you see, all pandas functions, such as the <span style="color: red;">iterrows()</span> method, are available in geopandas without the need to call pandas separately. Geopandas builds on top of pandas, and it inherits most of its functionality.

Of course the <span style="color: red;">iterrows()</span> pattern is not the most convenient and efficient way to calculate the area of many rows. Both <span style="color: red;">GeoSeries</span> (geometry columns) and <span style="color: red;">GeoDataFrames</span> have an <span style="color: red;">area</span> property:


There are other more efficient ways we can compute the area that are embedded into <span style="color: red;">GeoDataFrames</span> and <span style="color: red;">GeoSeries</span>

In [ ]:
# the `area` property of a `GeoDataFrame`
data_set.area[0]

In [ ]:
# Iterate over the first 5 rows of the data set
for i, row in data_set[:5].iterrows():
    polygon_area = data_set.area[i]
    print(f"The polygon in row {i} has a surface area of {polygon_area:0.1f} m².")

In [ ]:
#create an area column
data_set["area"] = data_set.area
data_set

Let's save a sub-set of this land use dataframe into another file.

Let's save water bodies (code 901)

In [ ]:
water_bodies=data_set[data_set.ID_CLASS== 901]

In [ ]:
water_bodies.plot()

In [ ]:
water_bodies.to_file(DATA_DIRECTORY  / "water_bodies.shp")

The groupby() function in (geo)pandas' data frames is an incredibly useful method. It enables splitting of data into groups based on specific criteria, applying functions individually to each group, and combining the results into a common data structure.

Let's find out how many classes this dataset has.

In [ ]:
data_set["CLASS_NAME"].unique()

Let's group them by their name.

In [ ]:
grouped_data = data_set.groupby("CLASS_NAME")
grouped_data

So, grouped_data is a DataFrameGroupBy object. Inside a <span style="color: red;">GroupBy</span> object, its property groups is a dictionary that works as a lookup table: it records which rows belong to which group. The keys of the dictionary are the unique values of the grouping column:

In [ ]:
grouped_data.groups

However, one can also simply iterate over the entire  <span style="color: red;">GroupBy</span> object. Let’s count how many rows of data each group has:

In [ ]:
for key, group in grouped_data:
    print(f"Terrain class {key} has {len(group)} rows.")

In [ ]:
water_bodies = grouped_data.get_group('Vattenyta')
type(water_bodies)
water_bodies.plot()

The index in the new data frame stays the same as in the ungrouped input data set. This can be helpful, for instance, when you want to join the grouped data back to the original input data.

In [ ]:
# Iterate over the input data, grouped by CLASS
for key, group in data_set.groupby("CLASS_NAME"):
    # save the group to a new shapefile
    group.to_file(DATA_DIRECTORY   / f"LU_{key}.shp")

### Extra: save summary statistics to CSV files

Whenever the results of an operation on a <span style="color: red;">GeoDataFrame.DataFrame</span> do not include a geometry, the output data frame will automatically become a ‘plain’ <span style="color: red;">pandas.DataFrame</span>, and can be saved to the standard table formats.

One interesting application of this is to save basic descriptive statistics of a geospatial data set into a CSV table. For instance, we might want to know the area each terrain class covers.

Again, we start by grouping the input data by terrain classes, and then compute the sum of each classes’ area. This can be condensed into one line of code:

In [ ]:
area_information = data_set.groupby("CLASS_NAME").area.sum()
area_information

We can then save the resulting table into a CSV file using the standard pandas approach 

In [ ]:
area_information.to_csv( "data/area_by_LU_class.csv")


## Sources

This lesson is inspired by the [Programming in Python lessons](http://swcarpentry.github.io/python-novice-inflammation/) from the [Software Carpentry organization](http://software-carpentry.org) and has adapted or reused material from University of Helsinki Automating GIS processis course (https://autogis-site.readthedocs.io/en/latest/course-info/license.html) under a Creative Commons Attribution-ShareAlike 4.0 International licence (https://creativecommons.org/licenses/by-sa/4.0/deed.en).